## 載入資料

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 超參數
num_epochs = 5
batch_size = 64
learning_rate = [0.001, 0.01, 0.1]
dropout_rates = [0.2, 0.5, 0.8]

# MNIST數據預處理
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
mnist_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)

# 分割數據為訓練集和測試集
train_size = int(0.8 * len(mnist_dataset))
test_size = len(mnist_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(mnist_dataset, [train_size, test_size])

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

## 建立模型

In [ ]:
# 定義具有Dropout的ANN模型
class ANNWithDropout(nn.Module):
    def __init__(self, dropout_rate):
        super(ANNWithDropout, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)  # 根據超參數添加Dropout
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)  # 在這一層後應用Dropout
        x = self.fc2(x)
        return x

## 使用 GridSearchCV 執行網格搜索

In [ ]:
# 定義網格搜索參數
param_grid = {'learning_rate': learning_rate, 'dropout_rate': dropout_rates}



# 使用 GridSearchCV 執行網格搜索
best_accuracy = 0
best_params = None

for lr in learning_rate:
    for dropout in dropout_rates:
        model = ANNWithDropout(dropout)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)

        model.train()
        for epoch in range(num_epochs):
            for images, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in test_loader:
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print(f'Learning Rate: {lr}, Dropout Rate: {dropout}, Test Accuracy: {accuracy:.4f}')

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {'learning_rate': lr, 'dropout_rate': dropout}

print("Best Parameters:", best_params)
print("Best Test Accuracy:", best_accuracy)
